## Step 1: Import Libraries


In [242]:
# ===== 필요한 라이브러리 import =====

# PyTorch 관련
import torch  # PyTorch 메인 라이브러리
import torch.nn as nn  # 신경망 모듈 (레이어, 활성화 함수 등)
import torch.optim as optim  # 최적화 알고리즘 (Adam, SGD 등)
import torch.nn.functional as F  # 함수형 API (softmax, relu 등)

# 데이터 처리 관련
import pandas as pd  # 데이터프레임 처리 (CSV 읽기 등)
import numpy as np  # 수치 연산
import re  # 정규표현식 (텍스트 전처리)
import math  # 수학 함수 (sin, cos 등)

# 토크나이저 및 데이터 로더
import sentencepiece as spm  # SentencePiece 토크나이저 (서브워드 단위)
from sklearn.model_selection import train_test_split  # 데이터를 train/val/test로 분할
from torch.utils.data import Dataset, DataLoader  # PyTorch 데이터셋 및 배치 로더

# 유틸리티
from tqdm import tqdm  # 진행바 표시용
import easydict  # 딕셔너리를 객체처럼 사용 가능 (config.emb_dim 형태)

# 디바이스 설정: Apple Silicon의 MPS 가속기 사용 가능하면 사용, 아니면 CPU
device = torch.device('mps' if torch.backends.mps.is_available() else 'cpu')
print(f'Using device: {device}')  # 현재 사용 중인 디바이스 출력


Using device: mps


## Step 2: Data Preprocessing

In [243]:
def preprocess_sentence(sentence):
    """
    단일 문장을 전처리하는 함수
    목적: 텍스트 데이터를 정제하여 모델 학습에 적합한 형태로 만듦
    """
    # 입력 검증: 비어있거나 None인 경우 빈 문자열 반환
    if pd.isna(sentence) or sentence is None:
        return ""

    # 문자열로 변환 (안전장치)
    sentence = str(sentence)

    # 정규표현식으로 필요한 문자만 남기기
    # ㄱ-ㅎ: 자음, ㅏ-ㅣ: 모음, 가-힣: 완성형 한글
    # a-zA-Z: 영어, 0-9: 숫자, \s: 공백
    # .,!?~: 문장부호, ㅠㅜ: 이모티콘
    # 나머지는 모두 공백으로 치환
    sentence = re.sub(r'[^ㄱ-ㅎㅏ-ㅣ가-힣a-zA-Z0-9\s.,!?~ㅠㅜ]', ' ', sentence) #re.sub는 정규표현식(regex)을 사용하여 특정 패턴의 문자를 찾아 다른 문자로 바꿉니다.

    # 연속된 여러 공백을 하나의 공백으로 통일
    sentence = re.sub(r'\s+', ' ', sentence)

    # 설명: \s는 공백, +는 "1개 이상"을 의미합니다. 즉, \s+는 "1개 이상의 연속된 공백"(예: " ", " ", " ")을 찾습니다.

    # 동작: 이렇게 찾은 연속된 공백 뭉치를 **하나의 공백(' ')**으로 압축합니다.

    # 이유: 바로 앞 단계에서 Hi!!^^가 Hi 처럼 여러 공백으로 바뀔 수 있습니다. 단어 사이처럼 불필요하게 공백이 많은 것은 단어 사이와 동일하게 취급되어야 합니다.

    # 문장 앞뒤 공백 제거
    sentence = sentence.strip()

    # 연속된 문장부호 정리 (예: !!! -> !, ??? -> ?)
    # ([!?.])를 캡처하고 \1+로 반복을 찾아서 r'\1'로 하나만 남김
    sentence = re.sub(r'([!?.])\1+', r'\1', sentence)

    return sentence


def load_and_preprocess_data(file_path):
    """
    CSV 파일에서 질문-답변 데이터를 로드하고 전처리
    """
    print("=" * 50)
    print("데이터 로드 및 전처리 중...")
    print("=" * 50)

    # pandas로 CSV 파일 읽기
    df = pd.read_csv(file_path)
    print(f"전체 데이터: {len(df)} 쌍")

    # 전처리된 질문과 답변을 저장할 리스트 초기화
    questions = []
    answers = []

    # 모든 질문-답변 쌍을 순회
    for i, (q, a) in enumerate(zip(df['Q'], df['A'])):
        # 각각 전처리 적용
        clean_q = preprocess_sentence(q)
        clean_a = preprocess_sentence(a)

        # 둘 다 유효한 문장인 경우만 저장
        if clean_q and clean_a:
            questions.append(clean_q)
            answers.append(clean_a)

        # 진행 상황 출력 (매 1000개마다)
        if (i + 1) % 1000 == 0:
            print(f"진행: {i + 1}/{len(df)}")

    print(f"\n전처리 후 유효한 쌍: {len(questions)}")
    print("\n샘플 데이터:")

    # 처음 3개의 샘플 데이터 출력하여 확인
    for i in range(min(3, len(questions))):
        print(f"Q: {questions[i]}")
        print(f"A: {answers[i]}\n")

    return questions, answers


In [244]:
# 데이터 로드 및 전처리 실행
# 파일 경로를 실제 데이터가 있는 위치로 수정 필요
file_path = '/Users/wansookim/Downloads/code_implementation/transformer_project_submit/data/ChatbotData.csv'

# 함수 호출하여 전처리된 질문과 답변 리스트 받기
questions, answers = load_and_preprocess_data(file_path)


데이터 로드 및 전처리 중...
전체 데이터: 11823 쌍
진행: 1000/11823
진행: 2000/11823
진행: 3000/11823
진행: 4000/11823
진행: 5000/11823
진행: 6000/11823
진행: 7000/11823
진행: 8000/11823
진행: 9000/11823
진행: 10000/11823
진행: 11000/11823

전처리 후 유효한 쌍: 11823

샘플 데이터:
Q: 12시 땡!
A: 하루가 또 가네요.

Q: 1지망 학교 떨어졌어
A: 위로해 드립니다.

Q: 3박4일 놀러가고 싶다
A: 여행은 언제나 좋죠.



## Step 3: SentencePiece Tokenization

In [245]:
def train_sentencepiece_model(questions, answers, model_prefix='/Users/wansookim/Downloads/code_implementation/transformer_project_submit', vocab_size=1200):
    """
    SentencePiece 모델 학습
    SentencePiece: 텍스트를 서브워드 단위로 분리하는 토크나이저
    장점: OOV(Out-of-Vocabulary) 문제 해결, 한국어에 효과적
    """
    print("=" * 50)
    print("SentencePiece 모델 학습 중...")
    print("=" * 50)

    # 모든 문장을 하나의 텍스트 파일로 저장 (SentencePiece 입력 형식)
    all_sentences_path = '/Users/wansookim/Downloads/code_implementation/transformer_project_submit/sentencepiece'
    with open(all_sentences_path, 'w', encoding='utf-8') as f:
        # 질문과 답변을 모두 합쳐서 줄바꿈으로 구분하여 저장
        f.write('\n'.join(questions + answers))

    # SentencePiece 학습 명령어 설정
    cmd = f'--input={all_sentences_path} \
           --model_prefix={model_prefix} \
           --vocab_size={vocab_size} \
           --model_type=unigram \
           --max_sentence_length=999999 \
           --pad_id=0 \
           --unk_id=1 \
           --bos_id=2 \
           --eos_id=3 \
           --user_defined_symbols=[SEP],[CLS],[MASK]'
    # --input: 학습 데이터 경로
    # --model_prefix: 저장될 모델 파일명 접두사
    # --vocab_size: 어휘 사전 크기 (8000개의 서브워드)
    # --model_type: unigram 언어 모델 사용
    # --pad_id: 패딩 토큰 ID (0)
    # --unk_id: 미등록 토큰 ID (1)
    # --bos_id: 문장 시작 토큰 ID (2)
    # --eos_id: 문장 종료 토큰 ID (3)

    # SentencePiece 모델 학습 실행
    spm.SentencePieceTrainer.Train(cmd)

    # 학습된 모델 파일 경로 생성
    model_file = f"{model_prefix}.model"
    print(f"\n모델 저장됨: {model_file}")
    return model_file


class SentencePieceVocab:
    """
    SentencePiece 모델 래퍼 클래스
    목적: SentencePiece 모델을 쉽게 사용하기 위한 인터페이스 제공
    """
    def __init__(self, sp_model_path):
        # SentencePiece 프로세서 초기화
        self.sp = spm.SentencePieceProcessor()
        # 학습된 모델 로드
        self.sp.Load(sp_model_path)

        # 특수 토큰 ID 정의
        self.PAD_ID = 0  # 패딩 (빈 공간 채우기)
        self.UNK_ID = 1  # 미등록 단어
        self.BOS_ID = 2  # 문장 시작 (Beginning Of Sentence)
        self.EOS_ID = 3  # 문장 끝 (End Of Sentence)

        # 토큰 문자열 -> ID 매핑
        self.stoi = {'<pad>': 0, '<unk>': 1, '<s>': 2, '</s>': 3}

        # ID -> 토큰 문자열 매핑 (전체 어휘)
        self.itos = [self.sp.IdToPiece(i) for i in range(self.sp.GetPieceSize())]

    def encode(self, sentence):
        """문장을 토큰 ID 리스트로 인코딩"""
        return self.sp.EncodeAsIds(sentence)

    def decode(self, ids):
        """
        토큰 ID 리스트를 문장으로 디코딩
        특수 토큰(pad, bos, eos)은 제외하고 디코딩
        """
        return self.sp.DecodeIds([i for i in ids if i not in [0, 2, 3]])

    def __len__(self):
        """어휘 사전 크기 반환"""
        return self.sp.GetPieceSize()


class ChatbotDataset(Dataset):
    """
    PyTorch Dataset 클래스
    목적: 질문-답변 쌍을 PyTorch 모델에 입력 가능한 형태로 변환
    """
    def __init__(self, questions, answers, vocab, max_length=40):
        # 데이터 저장
        self.questions = questions
        self.answers = answers
        self.vocab = vocab  # SentencePiece vocab 객체
        self.max_length = max_length  # 최대 시퀀스 길이 (잘림 방지)

    def __len__(self):
        """데이터셋 크기 반환"""
        return len(self.questions)

    def __getitem__(self, idx):
        """
        특정 인덱스의 데이터를 가져옴
        반환: 토큰화된 질문(SRC)과 답변(TRG)
        """
        # idx번째 질문과 답변 가져오기
        q = self.questions[idx]
        a = self.answers[idx]

        # 질문 토큰화: [BOS] + 토큰들 + [EOS]
        src = [self.vocab.BOS_ID] + self.vocab.encode(q) + [self.vocab.EOS_ID]

        # 답변 토큰화: [BOS] + 토큰들 + [EOS]
        trg = [self.vocab.BOS_ID] + self.vocab.encode(a) + [self.vocab.EOS_ID]

        # PyTorch tensor로 변환 (dtype=long은 정수형, embedding 입력으로 필요)
        return {
            'SRC': torch.tensor(src[:self.max_length], dtype=torch.long),  # 최대 길이로 자르기
            'TRG': torch.tensor(trg[:self.max_length], dtype=torch.long)
        }


def collate_fn(batch, pad_idx=0):
    """
    DataLoader의 배치 생성 함수
    목적: 서로 다른 길이의 시퀀스를 같은 길이로 패딩하여 배치 생성
    """
    # 배치에서 SRC(질문)와 TRG(답변) 분리
    src_batch = [item['SRC'] for item in batch]
    trg_batch = [item['TRG'] for item in batch]

    # 배치 내 최대 길이 찾기
    src_max = max(len(s) for s in src_batch)
    trg_max = max(len(t) for t in trg_batch)

    # 각 시퀀스를 최대 길이에 맞춰 패딩
    # 짧은 시퀀스는 뒤에 pad_idx(0)를 채워서 같은 길이로 만듦
    # dtype=torch.long 명시: embedding 레이어는 정수형 입력 필요
    src_padded = [torch.cat([s, torch.tensor([pad_idx]*(src_max-len(s)), dtype=torch.long)])
                  for s in src_batch]
    trg_padded = [torch.cat([t, torch.tensor([pad_idx]*(trg_max-len(t)), dtype=torch.long)])
                  for t in trg_batch]

    # 리스트의 텐서들을 하나의 텐서로 쌓기 (batch_size, seq_len)
    return {'SRC': torch.stack(src_padded), 'TRG': torch.stack(trg_padded)}


In [246]:
# ===== SentencePiece 모델 학습 및 어휘 사전 생성 =====

# SentencePiece 모델 학습 실행
model_file = train_sentencepiece_model(questions, answers)

# Vocab 객체 생성 (토크나이저 래퍼)
vocab = SentencePieceVocab(model_file)
print(f"\n어휘 사전 크기: {len(vocab):,}")  # 어휘에 포함된 총 토큰 수

# 토크나이징 테스트
test_sentence = questions[0]  # 첫 번째 질문으로 테스트
encoded = vocab.encode(test_sentence)  # 문장 -> 토큰 ID로 인코딩
decoded = vocab.decode(encoded)  # 토큰 ID -> 문장으로 디코딩

# 인코딩/디코딩 결과 확인
print(f"\n테스트 문장: {test_sentence}")
print(f"인코딩 결과 (처음 10개): {encoded[:10]}...")  # 토큰 ID 리스트
print(f"디코딩 결과: {decoded}")  # 다시 문장으로 변환


SentencePiece 모델 학습 중...

모델 저장됨: /Users/wansookim/Downloads/code_implementation/transformer_project_submit.model

어휘 사전 크기: 1,200

테스트 문장: 12시 땡!
인코딩 결과 (처음 10개): [7, 680, 311, 55, 7, 1022, 115]...
디코딩 결과: 12시 땡!


sentencepiece_trainer.cc(178) LOG(INFO) Running command: --input=/Users/wansookim/Downloads/code_implementation/transformer_project_submit/sentencepiece            --model_prefix=/Users/wansookim/Downloads/code_implementation/transformer_project_submit            --vocab_size=1200            --model_type=unigram            --max_sentence_length=999999            --pad_id=0            --unk_id=1            --bos_id=2            --eos_id=3            --user_defined_symbols=[SEP],[CLS],[MASK]
sentencepiece_trainer.cc(78) LOG(INFO) Starts training with : 
trainer_spec {
  input: /Users/wansookim/Downloads/code_implementation/transformer_project_submit/sentencepiece
  input_format: 
  model_prefix: /Users/wansookim/Downloads/code_implementation/transformer_project_submit
  model_type: UNIGRAM
  vocab_size: 1200
  self_test_sample_size: 0
  character_coverage: 0.9995
  input_sentence_size: 0
  shuffle_input_sentence: 1
  seed_sentencepiece_size: 1000000
  shrinking_factor: 0.75
  max_sentenc

In [247]:
# ===== 데이터 분할: Train / Validation / Test =====

# 1단계: 전체 데이터를 Train(80%) + Temp(20%)로 분할
train_q, temp_q, train_a, temp_a = train_test_split(
    questions, answers,  # 전체 데이터
    test_size=0.2,  # 20%를 temp로
    random_state=42  # 재현성을 위한 랜덤 시드
)

# 2단계: Temp 데이터를 Validation(10%) + Test(10%)로 분할
val_q, test_q, val_a, test_a = train_test_split(
    temp_q, temp_a,  # temp 데이터 (전체의 20%)
    test_size=0.5,  # temp의 50% = 전체의 10%
    random_state=42
)

# PyTorch Dataset 객체 생성
train_dataset = ChatbotDataset(train_q, train_a, vocab, max_length=40)
val_dataset = ChatbotDataset(val_q, val_a, vocab, max_length=40)
test_dataset = ChatbotDataset(test_q, test_a, vocab, max_length=40)

# DataLoader 생성 (배치 단위로 데이터 로드)
train_iterator = DataLoader(
    train_dataset,
    batch_size=64,  # 한 번에 32개씩 처리
    shuffle=True,  # 학습 시 데이터 섞기 (과적합 방지)
    collate_fn=lambda b: collate_fn(b, vocab.PAD_ID)  # 패딩 적용
)

valid_iterator = DataLoader(
    val_dataset,
    batch_size=32,
    shuffle=False,  # 검증 시에는 섞지 않음
    collate_fn=lambda b: collate_fn(b, vocab.PAD_ID)
)

test_iterator = DataLoader(
    test_dataset,
    batch_size=32,
    shuffle=False,  # 테스트 시에도 섞지 않음
    collate_fn=lambda b: collate_fn(b, vocab.PAD_ID)
)

# 데이터 분할 결과 출력
print("=" * 50)
print("데이터 분할 완료")
print("=" * 50)
print(f"Train: {len(train_q):,} 쌍")  # 학습용
print(f"Val: {len(val_q):,} 쌍")  # 검증용 (하이퍼파라미터 튜닝)
print(f"Test: {len(test_q):,} 쌍")  # 최종 평가용


데이터 분할 완료
Train: 9,458 쌍
Val: 1,182 쌍
Test: 1,183 쌍


## Step 4: Model Building

In [248]:
class MultiHeadAttention(nn.Module):
    """
    멀티 헤드 어텐션 (Multi-Head Attention)
    논문: 'Attention Is All You Need' (Vaswani et al., 2017)

    핵심 아이디어:
    - 여러 개의 어텐션을 병렬로 수행 (다양한 관점에서 정보 추출)
    - 각 헤드는 서로 다른 표현 부분공간에 집중
    - 모든 헤드의 출력을 합쳐서 최종 출력 생성
    """

    def __init__(self, emb_dim, num_heads, dropout=0.0, bias=False,
                 encoder_decoder_attention=False, causal=False):
        super().__init__()

        # 하이퍼파라미터 저장
        self.emb_dim = emb_dim  # 임베딩 차원 (전체 모델의 차원)
        self.num_heads = num_heads  # 어텐션 헤드 개수
        self.dropout = dropout  # 드롭아웃 비율
        self.head_dim = emb_dim // num_heads  # 각 헤드의 차원

        # emb_dim이 num_heads로 나누어 떨어지는지 확인
        # 예: emb_dim=64, num_heads=8 → head_dim=8 (OK)
        assert self.head_dim * num_heads == self.emb_dim, "emb_dim은 num_heads로 나누어떨어져야 함"

        # 어텐션 타입 설정
        self.encoder_decoder_attention = encoder_decoder_attention  # 인코더-디코더 간 어텐션 여부
        self.causal = causal  # 인과적 마스킹 여부 (미래 토큰 참조 금지)

        # Query, Key, Value 투영 레이어
        # 입력을 Q, K, V로 변환하는 선형 변환
        self.q_proj = nn.Linear(emb_dim, emb_dim, bias=bias)  # Query 투영
        self.k_proj = nn.Linear(emb_dim, emb_dim, bias=bias)  # Key 투영
        self.v_proj = nn.Linear(emb_dim, emb_dim, bias=bias)  # Value 투영

        # 최종 출력 투영 레이어
        self.out_proj = nn.Linear(emb_dim, emb_dim, bias=bias)

    def transpose_for_scores(self, x):
        """
        텐서를 멀티 헤드 어텐션 계산에 적합한 형태로 변환

        입력 형태: (batch_size, seq_len, emb_dim)
        출력 형태: (batch_size, num_heads, seq_len, head_dim)

        이렇게 하면 각 헤드가 독립적으로 어텐션을 계산할 수 있음
        """
        # (batch_size, seq_len, emb_dim) → (batch_size, seq_len, num_heads, head_dim)
        new_x_shape = x.size()[:-1] + (self.num_heads, self.head_dim,)
        x = x.view(*new_x_shape)

        # (batch_size, seq_len, num_heads, head_dim) → (batch_size, num_heads, seq_len, head_dim)
        # permute로 헤드 차원을 앞으로 이동
        return x.permute(0, 2, 1, 3)

    def forward(self, query, key, attention_mask=None):
        """
        Forward pass: 어텐션 메커니즘 수행

        Args:
            query: 쿼리 텐서 (무엇을 찾고 싶은지)
            key: 키/밸류 텐서 (어디서 정보를 가져올지)
            attention_mask: 어텐션 마스크 (특정 위치 참조 금지)
        """
        # Query 투영: (batch, seq_len, emb_dim)
        q = self.q_proj(query)

        if self.encoder_decoder_attention:
            # 크로스 어텐션: Query는 디코더, Key/Value는 인코더에서
            k = self.k_proj(key)
            v = self.v_proj(key)
        else:
            # 셀프 어텐션: Query, Key, Value 모두 같은 입력에서
            k = self.k_proj(query)
            v = self.v_proj(query)

        # 멀티 헤드 형태로 변환
        q = self.transpose_for_scores(q)  # (batch, num_heads, seq_len, head_dim)
        k = self.transpose_for_scores(k)
        v = self.transpose_for_scores(v)

        # 어텐션 스코어 계산: Q와 K의 내적
        # (batch, num_heads, seq_len_q, head_dim) @ (batch, num_heads, head_dim, seq_len_k)
        # → (batch, num_heads, seq_len_q, seq_len_k)
        attn_weights = torch.matmul(q, k.transpose(-1, -2)) / math.sqrt(self.head_dim)
        # sqrt(head_dim)으로 나누는 이유: 스케일 조정 (그래디언트 안정화)

        # 어텐션 마스크 적용 (필요한 경우)
        if attention_mask is not None:
            if self.causal:
                # 인과적 마스킹: 미래 토큰을 볼 수 없도록 (디코더 self-attention)
                # -inf로 설정하면 softmax 후 0이 됨
                attn_weights = attn_weights.masked_fill(
                    attention_mask.unsqueeze(0).unsqueeze(1), float("-inf")
                )
            else:
                # 패딩 마스킹: 패딩 토큰을 참조하지 않도록
                attn_weights = attn_weights.masked_fill(
                    attention_mask.unsqueeze(1).unsqueeze(2), float("-inf")
                )

        # Softmax로 어텐션 확률 계산 (각 위치에 얼마나 집중할지)
        attn_weights = F.softmax(attn_weights, dim=-1)  # 합이 1이 되도록

        # 드롭아웃 적용 (학습 시 정규화)
        attn_probs = F.dropout(attn_weights, p=self.dropout, training=self.training)

        # 어텐션 가중치와 Value의 가중합 계산
        # (batch, num_heads, seq_len, seq_len) @ (batch, num_heads, seq_len, head_dim)
        # → (batch, num_heads, seq_len, head_dim)
        attn_output = torch.matmul(attn_probs, v)

        # 원래 형태로 되돌리기
        # (batch, num_heads, seq_len, head_dim) → (batch, seq_len, num_heads, head_dim)
        attn_output = attn_output.permute(0, 2, 1, 3).contiguous()

        # 모든 헤드를 연결 (concatenate)
        # (batch, seq_len, num_heads, head_dim) → (batch, seq_len, emb_dim)
        concat_attn_output_shape = attn_output.size()[:-2] + (self.emb_dim,)
        attn_output = attn_output.view(*concat_attn_output_shape)

        # 최종 선형 투영
        attn_output = self.out_proj(attn_output)

        # 출력: 어텐션 결과와 어텐션 가중치 (시각화용)
        return attn_output, attn_weights


In [249]:
class PositionWiseFeedForward(nn.Module):
    """
    위치별 피드포워드 네트워크 (Position-wise Feed-Forward Network)

    Transformer의 각 위치(토큰)에 독립적으로 적용되는 2층 fully-connected 네트워크
    역할: 어텐션으로 모은 정보를 비선형 변환하여 더 풍부한 표현 학습

    구조: Linear → ReLU → Dropout → Linear → Dropout → Residual Connection
    """

    def __init__(self, emb_dim, d_ff, dropout=0.1):
        super().__init__()

        # 첫 번째 선형 레이어: emb_dim → d_ff (차원 확장)
        # 보통 d_ff = 4 * emb_dim (예: 64 → 256)
        self.w_1 = nn.Linear(emb_dim, d_ff)

        # 두 번째 선형 레이어: d_ff → emb_dim (차원 축소)
        self.w_2 = nn.Linear(d_ff, emb_dim)

        self.dropout = dropout  # 드롭아웃 비율
        self.activation = nn.ReLU()  # 활성화 함수 (비선형성 추가)

    def forward(self, x):
        """
        Forward pass

        입력: (batch_size, seq_len, emb_dim)
        출력: (batch_size, seq_len, emb_dim)
        """
        # Residual connection을 위해 입력 저장
        residual = x

        # 1. 차원 확장 및 활성화
        x = self.activation(self.w_1(x))  # (batch, seq_len, emb_dim) → (batch, seq_len, d_ff)

        # 2. 드롭아웃 (학습 시 정규화)
        x = F.dropout(x, p=self.dropout, training=self.training)

        # 3. 차원 축소 (원래 크기로)
        x = self.w_2(x)  # (batch, seq_len, d_ff) → (batch, seq_len, emb_dim)

        # 4. 드롭아웃
        x = F.dropout(x, p=self.dropout, training=self.training)

        # 5. Residual connection (입력을 출력에 더함)
        # 이유: 그래디언트 소실 방지, 학습 안정화
        return x + residual


In [250]:
class SinusoidalPositionalEmbedding(nn.Embedding):
    """
    사인/코사인 위치 임베딩 (Sinusoidal Positional Embedding)

    목적: 토큰의 순서 정보를 모델에 제공
    이유: Transformer는 순서를 고려하지 않는 구조이므로, 위치 정보를 명시적으로 추가해야 함

    수식:
    PE(pos, 2i) = sin(pos / 10000^(2i/d_model))
    PE(pos, 2i+1) = cos(pos / 10000^(2i/d_model))

    특징:
    - 학습 불필요 (고정된 수식)
    - 서로 다른 주기의 sin/cos 조합으로 고유한 위치 표현
    - 상대적 위치 관계도 학습 가능
    """

    def __init__(self, num_positions, embedding_dim, padding_idx=None):
        """
        Args:
            num_positions: 최대 위치 개수 (최대 시퀀스 길이)
            embedding_dim: 임베딩 차원
            padding_idx: 패딩 인덱스 (사용 안 함)
        """
        # 부모 클래스(nn.Embedding) 초기화
        super().__init__(num_positions, embedding_dim)

        # weight 파라미터를 사인/코사인 값으로 초기화
        self._init_weight(self.weight)

    @staticmethod
    def _init_weight(out):
        """
        위치 임베딩을 사인/코사인 패턴으로 초기화

        Args:
            out: 초기화할 파라미터 텐서 (num_positions, embedding_dim)
        """
        n_pos, embed_dim = out.shape  # 위치 개수, 임베딩 차원

        # 빈 텐서 생성 (임시)
        position_enc = torch.zeros(n_pos, embed_dim)

        # 각 위치에 대해
        for pos in range(n_pos):
            # 짝수 인덱스는 sin, 홀수 인덱스는 cos
            for i in range(0, embed_dim, 2):  # 2씩 증가 (짝수만)
                # 짝수 인덱스: sin 함수
                position_enc[pos, i] = math.sin(pos / (10000 ** (i / embed_dim)))

                # 홀수 인덱스: cos 함수 (경계 체크)
                if i + 1 < embed_dim:
                    position_enc[pos, i + 1] = math.cos(pos / (10000 ** ((i + 1) / embed_dim)))

        # 계산된 값을 파라미터의 data에 복사 (in-place)
        # .data.copy_()를 사용하면 autograd 그래프를 건드리지 않음
        out.data.copy_(position_enc)

        # 위치 임베딩은 학습하지 않음 (고정)
        out.requires_grad = False

    @torch.no_grad()  # 그래디언트 계산 불필요
    def forward(self, input_ids):
        """
        Forward pass: 입력 시퀀스 길이에 맞는 위치 임베딩 반환

        Args:
            input_ids: 입력 토큰 ID (batch_size, seq_len)

        Returns:
            위치 임베딩: (1, seq_len, embedding_dim) 또는 (seq_len, embedding_dim)
        """
        bsz, seq_len = input_ids.shape[:2]  # 배치 크기, 시퀀스 길이

        # 0부터 seq_len-1까지의 위치 인덱스 생성
        # device를 weight와 같은 곳에 두기 (GPU/CPU 일치)
        positions = torch.arange(seq_len, dtype=torch.long, device=self.weight.device)

        # nn.Embedding의 forward를 호출하여 위치 임베딩 가져오기
        return super().forward(positions)


In [251]:
class EncoderLayer(nn.Module):
    """
    Transformer 인코더의 단일 레이어

    구조:
    1. Multi-Head Self-Attention
    2. Layer Normalization
    3. Position-wise Feed-Forward
    4. Layer Normalization

    각 서브레이어 후에 residual connection 적용
    """

    def __init__(self, config):
        super().__init__()

        # 셀프 어텐션 레이어
        # 입력 시퀀스의 각 토큰이 다른 모든 토큰과 상호작용
        self.self_attn = MultiHeadAttention(
            emb_dim=config.emb_dim,  # 임베딩 차원
            num_heads=config.num_heads,  # 어텐션 헤드 개수
            dropout=config.attention_dropout,  # 어텐션 드롭아웃
            causal=False  # 인코더는 양방향 (모든 토큰 참조 가능)
        )

        # 레이어 정규화 (Layer Normalization)
        # 각 샘플의 특성들을 정규화 (배치 정규화와 다름)
        self.norm1 = nn.LayerNorm(config.emb_dim)

        # 피드포워드 네트워크
        self.ffn = PositionWiseFeedForward(
            emb_dim=config.emb_dim,
            d_ff=config.ffn_dim,  # 중간 확장 차원 (보통 4*emb_dim)
            dropout=config.dropout
        )

        # 두 번째 레이어 정규화
        self.norm2 = nn.LayerNorm(config.emb_dim)

        self.dropout = config.dropout

    def forward(self, x, attention_mask=None):
        """
        Forward pass

        Args:
            x: 입력 (batch_size, seq_len, emb_dim)
            attention_mask: 패딩 마스크 (batch_size, seq_len)
        """
        # 1. Multi-Head Self-Attention
        # 입력을 Query, Key, Value로 모두 사용 (셀프 어텐션)
        attn_output, attn_weights = self.self_attn(
            query=x,  # Query
            key=x,  # Key = Query (셀프 어텐션)
            attention_mask=attention_mask  # 패딩 위치 마스킹
        )

        # Residual connection + Dropout
        x = x + F.dropout(attn_output, p=self.dropout, training=self.training)

        # Layer Normalization (Post-LN 방식)
        x = self.norm1(x)

        # 2. Position-wise Feed-Forward Network
        # (내부에 이미 residual connection 포함)
        x = self.ffn(x)

        # Layer Normalization
        x = self.norm2(x)

        return x, attn_weights  # 출력과 어텐션 가중치 반환


In [252]:
class Encoder(nn.Module):
    """
    Transformer 인코더

    입력 시퀀스를 받아서 문맥을 이해한 표현(representation)으로 변환
    여러 개의 EncoderLayer를 쌓아서 구성

    처리 과정:
    1. 토큰 임베딩 + 위치 임베딩
    2. N개의 EncoderLayer 통과
    3. 최종 인코딩된 표현 출력
    """

    def __init__(self, config, embed_tokens):
        super().__init__()

        # 패딩 인덱스 저장 (어텐션에서 마스킹할 때 사용)
        self.padding_idx = embed_tokens.padding_idx

        # 토큰 임베딩 레이어 (단어 -> 벡터)
        self.embed_tokens = embed_tokens

        # 위치 임베딩 레이어 (순서 정보 제공)
        self.embed_positions = SinusoidalPositionalEmbedding(
            config.max_position_embeddings,  # 최대 시퀀스 길이
            config.emb_dim,  # 임베딩 차원
            self.padding_idx
        )

        # N개의 EncoderLayer 스택
        # ModuleList: 여러 레이어를 리스트로 관리 (자동으로 파라미터 등록)
        self.layers = nn.ModuleList([
            EncoderLayer(config) for _ in range(config.encoder_layers)
        ])

    def forward(self, input_ids, attention_mask=None):
        """
        Forward pass

        Args:
            input_ids: 입력 토큰 ID (batch_size, seq_len)
            attention_mask: 패딩 마스크 (batch_size, seq_len)

        Returns:
            인코딩된 표현 (batch_size, seq_len, emb_dim)
            어텐션 스코어 리스트
        """
        # 1. 토큰 임베딩: 토큰 ID -> 임베딩 벡터
        # (batch_size, seq_len) → (batch_size, seq_len, emb_dim)
        inputs_embeds = self.embed_tokens(input_ids)

        # 2. 위치 임베딩: 각 위치의 위치 정보
        # (seq_len, emb_dim)
        embed_pos = self.embed_positions(input_ids)

        # 3. 토큰 임베딩 + 위치 임베딩
        # 두 임베딩을 더해서 최종 입력 표현 생성
        x = inputs_embeds + embed_pos

        # 어텐션 스코어를 저장할 리스트 (시각화나 분석용)
        attention_scores = []

        # 4. 모든 EncoderLayer를 순차적으로 통과
        for layer in self.layers:
            # 각 레이어의 출력이 다음 레이어의 입력이 됨
            x, attn = layer(x, attention_mask=attention_mask)
            attention_scores.append(attn)  # 각 레이어의 어텐션 저장

        # 최종 인코딩된 표현과 모든 레이어의 어텐션 반환
        return x, attention_scores


In [253]:
class DecoderLayer(nn.Module):
    """
    Transformer 디코더의 단일 레이어

    구조 (3개의 서브레이어):
    1. Masked Multi-Head Self-Attention (미래 토큰 참조 불가)
    2. Multi-Head Cross-Attention (인코더 출력 참조)
    3. Position-wise Feed-Forward Network

    각 서브레이어마다 residual connection + layer normalization
    """

    def __init__(self, config):
        super().__init__()

        # 1. Masked Self-Attention
        # 디코더의 각 위치는 이전 위치들만 참조 가능 (causal=True)
        self.self_attn = MultiHeadAttention(
            emb_dim=config.emb_dim,
            num_heads=config.num_heads,
            dropout=config.attention_dropout,
            causal=True  # 미래 토큰 마스킹
        )
        self.norm1 = nn.LayerNorm(config.emb_dim)

        # 2. Cross-Attention (Encoder-Decoder Attention)
        # Query: 디코더, Key/Value: 인코더 출력
        self.cross_attn = MultiHeadAttention(
            emb_dim=config.emb_dim,
            num_heads=config.num_heads,
            dropout=config.attention_dropout,
            encoder_decoder_attention=True,  # 크로스 어텐션 활성화
            causal=False  # 인코더 출력은 모두 참조 가능
        )
        self.norm2 = nn.LayerNorm(config.emb_dim)

        # 3. Feed-Forward Network
        self.ffn = PositionWiseFeedForward(
            emb_dim=config.emb_dim,
            d_ff=config.ffn_dim,
            dropout=config.dropout
        )
        self.norm3 = nn.LayerNorm(config.emb_dim)

        self.dropout = config.dropout

    def forward(self, x, encoder_output, encoder_attention_mask=None, decoder_causal_mask=None):
        """
        Forward pass

        Args:
            x: 디코더 입력 (batch_size, tgt_len, emb_dim)
            encoder_output: 인코더 출력 (batch_size, src_len, emb_dim)
            encoder_attention_mask: 인코더 패딩 마스크
            decoder_causal_mask: 디코더 인과 마스크 (미래 마스킹)
        """
        # 1. Masked Self-Attention
        # 현재까지 생성된 토큰들끼리만 어텐션
        self_attn_output, self_attn_weights = self.self_attn(
            query=x,
            key=x,  # 셀프 어텐션
            attention_mask=decoder_causal_mask  # 미래 마스킹
        )

        # Residual + Dropout + LayerNorm
        x = x + F.dropout(self_attn_output, p=self.dropout, training=self.training)
        x = self.norm1(x)

        # 2. Cross-Attention
        # Query: 디코더, Key/Value: 인코더
        # 디코더가 인코더의 정보를 참조
        cross_attn_output, cross_attn_weights = self.cross_attn(
            query=x,  # 디코더의 현재 상태
            key=encoder_output,  # 인코더의 출력 (소스 문장 정보)
            attention_mask=encoder_attention_mask  # 소스 패딩 마스킹
        )

        # Residual + Dropout + LayerNorm
        x = x + F.dropout(cross_attn_output, p=self.dropout, training=self.training)
        x = self.norm2(x)

        # 3. Feed-Forward Network
        x = self.ffn(x)
        x = self.norm3(x)

        # 출력과 어텐션 가중치 반환
        return x, (self_attn_weights, cross_attn_weights)


In [254]:
class Decoder(nn.Module):
    """
    Transformer 디코더

    역할:
    - 인코더의 출력을 참조하면서 순차적으로 답변 생성
    - 이전에 생성한 토큰들과 인코더 정보를 활용

    처리 과정:
    1. 타겟 토큰 임베딩 + 위치 임베딩
    2. N개의 DecoderLayer 통과
    3. 다음 토큰 예측을 위한 표현 출력
    """

    def __init__(self, config, embed_tokens):
        super().__init__()

        # 패딩 인덱스
        self.padding_idx = embed_tokens.padding_idx

        # 타겟 토큰 임베딩 (답변 단어 -> 벡터)
        self.embed_tokens = embed_tokens

        # 위치 임베딩
        self.embed_positions = SinusoidalPositionalEmbedding(
            config.max_position_embeddings,
            config.emb_dim,
            self.padding_idx
        )

        # N개의 DecoderLayer 스택
        self.layers = nn.ModuleList([
            DecoderLayer(config) for _ in range(config.decoder_layers)
        ])

    def forward(self, input_ids, encoder_output,
                encoder_attention_mask=None, decoder_causal_mask=None):
        """
        Forward pass

        Args:
            input_ids: 타겟 토큰 ID (batch_size, tgt_len)
            encoder_output: 인코더 출력 (batch_size, src_len, emb_dim)
            encoder_attention_mask: 소스 패딩 마스크
            decoder_causal_mask: 디코더 인과 마스크

        Returns:
            디코더 출력 (batch_size, tgt_len, emb_dim)
            어텐션 스코어 리스트
        """
        # 1. 타겟 토큰 임베딩
        inputs_embeds = self.embed_tokens(input_ids)

        # 2. 위치 임베딩
        embed_pos = self.embed_positions(input_ids)

        # 3. 토큰 임베딩 + 위치 임베딩
        x = inputs_embeds + embed_pos

        # 어텐션 스코어 저장
        attention_scores = []

        # 4. 모든 DecoderLayer 통과
        for layer in self.layers:
            # 각 레이어는 셀프 어텐션 + 크로스 어텐션 + FFN 수행
            x, attn = layer(
                x,  # 디코더 입력
                encoder_output,  # 인코더 출력 (소스 정보)
                encoder_attention_mask=encoder_attention_mask,
                decoder_causal_mask=decoder_causal_mask
            )
            attention_scores.append(attn)  # (self_attn, cross_attn) 튜플

        return x, attention_scores


In [255]:
class Transformer(nn.Module):
    """
    완전한 Transformer 모델 (Sequence-to-Sequence)

    구조:
    1. 소스/타겟 임베딩 레이어
    2. Encoder (입력 인코딩)
    3. Decoder (출력 생성)
    4. 최종 출력 레이어 (어휘 확률 분포)

    사용 예:
    - 기계 번역 (영어 -> 한국어)
    - 대화 시스템 (질문 -> 답변)
    - 요약, 생성 등
    """

    def __init__(self, vocab, config):
        super().__init__()

        # 설정 저장
        self.vocab = vocab  # 어휘 사전
        self.config = config

        # 소스(입력) 임베딩 레이어
        # len(vocab.itos): 전체 어휘 크기
        # padding_idx: 패딩 토큰은 임베딩하지 않음 (그래디언트 0)
        self.enc_embedding = nn.Embedding(
            len(vocab.itos), config.emb_dim, padding_idx=vocab.stoi['<pad>']
        )

        # 타겟(출력) 임베딩 레이어
        self.dec_embedding = nn.Embedding(
            len(vocab.itos), config.emb_dim, padding_idx=vocab.stoi['<pad>']
        )

        # Encoder와 Decoder 초기화
        self.encoder = Encoder(config, self.enc_embedding)
        self.decoder = Decoder(config, self.dec_embedding)

        # 출력 레이어: 임베딩 차원 -> 어휘 크기
        # 각 위치에서 다음 토큰의 확률 분포 생성
        self.prediction_head = nn.Linear(config.emb_dim, len(vocab.itos))

    def generate_mask(self, src, trg):
        """
        어텐션 마스크 생성

        1. 인코더 패딩 마스크: 소스 문장의 패딩 위치 표시
        2. 디코더 인과 마스크: 미래 토큰 참조 방지

        Args:
            src: 소스 토큰 ID (batch_size, src_len)
            trg: 타겟 토큰 ID (batch_size, tgt_len)
        """
        # 1. 인코더 패딩 마스크
        # 패딩 토큰(0)인 위치를 True로 표시
        # True인 위치는 어텐션 계산 시 무시됨
        enc_attention_mask = (src == self.vocab.stoi['<pad>'])

        # 2. 디코더 인과 마스크 (Causal Mask)
        # 각 위치에서 이후 위치를 볼 수 없도록 상삼각 행렬 생성
        tgt_len = trg.size(1)
        # torch.triu: 상삼각 행렬 (대각선 위쪽만 1)
        # diagonal=1: 대각선 바로 위부터
        # 예: [[0, 1, 1],
        #      [0, 0, 1],
        #      [0, 0, 0]]
        dec_causal_mask = torch.triu(
            torch.ones(tgt_len, tgt_len, dtype=torch.bool, device=trg.device),
            diagonal=1
        )
        # True인 위치는 참조 불가 (미래 토큰)

        return enc_attention_mask, dec_causal_mask

    def forward(self, src, trg):
        """
        Forward pass: 소스에서 타겟으로 변환

        Args:
            src: 소스 시퀀스 (batch_size, src_len) - 질문
            trg: 타겟 시퀀스 (batch_size, tgt_len) - 답변

        Returns:
            output: 예측 로짓 (batch_size, tgt_len, vocab_size)
            encoder_attention_scores: 인코더 어텐션
            decoder_attention_scores: 디코더 어텐션
        """
        # 1. 마스크 생성
        enc_attention_mask, dec_causal_mask = self.generate_mask(src, trg)

        # 2. 인코더: 소스 문장 인코딩
        # 입력 질문을 이해한 표현으로 변환
        encoder_output, encoder_attention_scores = self.encoder(
            input_ids=src,
            attention_mask=enc_attention_mask
        )

        # 3. 디코더: 타겟 문장 생성
        # 인코더 출력을 참조하면서 답변 생성
        decoder_output, decoder_attention_scores = self.decoder(
            trg,  # 타겟 입력 (teacher forcing)
            encoder_output,  # 인코더 출력 (소스 정보)
            encoder_attention_mask=enc_attention_mask,  # 소스 패딩
            decoder_causal_mask=dec_causal_mask,  # 미래 마스킹
        )

        # 4. 최종 출력: 어휘 확률 분포
        # (batch_size, tgt_len, emb_dim) → (batch_size, tgt_len, vocab_size)
        decoder_output = self.prediction_head(decoder_output)

        return decoder_output, encoder_attention_scores, decoder_attention_scores


In [256]:
# ===== 모델 설정 및 초기화 =====

# 하이퍼파라미터 설정
config = easydict.EasyDict({
    # 모델 차원
    "emb_dim": 256,  # 임베딩 차원 (단어를 64차원 벡터로 표현)
    "ffn_dim": 1024,  # Feed-Forward 중간 차원 (보통 emb_dim의 4배)

    # 어텐션 설정
    "num_heads": 8,  # 멀티 헤드 어텐션의 헤드 개수
    "attention_dropout": 0.1,  # 어텐션 가중치 드롭아웃

    # 레이어 개수
    "encoder_layers": 3,  # 인코더 레이어 수 (더 깊을수록 표현력 증가)
    "decoder_layers": 3,  # 디코더 레이어 수

    # 기타
    "dropout": 0.35,  # 일반 드롭아웃 (과적합 방지)
    "max_position_embeddings": 40  # 최대 시퀀스 길이
})

# 모델 생성
model = Transformer(vocab, config)

# 모델을 디바이스로 이동 (GPU/MPS 사용 가능하면 사용)
model.to(device)

# 옵티마이저: Adam (학습률 0.001)
# Adam은 학습률을 자동으로 조정하는 효율적인 최적화 알고리즘
# optimizer = optim.AdamW(
#     model.parameters(),
#     lr=0.0005,
#     betas=(0.9, 0.98),      # Standard for Transformers
#     eps=1e-9,
#     weight_decay=0.01       # Proper weight decay
# )

optimizer = optim.AdamW(
    model.parameters(),
    lr=0.0005,          # 0.0005 → 0.0003
    betas=(0.9, 0.98),
    eps=1e-9,
    weight_decay=0.01    # 0.01 → 0.1
)

# 손실 함수: Cross Entropy Loss
# ignore_index: 패딩 토큰은 손실 계산에서 제외
criterion = nn.CrossEntropyLoss(ignore_index=vocab.stoi['<pad>'])

# 그래디언트 클리핑 값 (그래디언트 폭발 방지)
CLIP = 1.0

# 학습 에폭 수
N_EPOCHS = 200

# Early Stopping 설정
best_valid_loss = float('inf')  # 최고 검증 손실 초기화
patience = 5  # 성능 개선이 없으면 3 에폭 후 조기 종료
patience_counter = 0  # 카운터 초기화

print(f"모델 파라미터 수: {sum(p.numel() for p in model.parameters()):,}")


모델 파라미터 수: 6,463,664


## Step 5: Model Training and Evaluation

In [257]:
def train(model, iterator, optimizer, criterion, clip):
    """
    한 에폭 동안 모델 학습

    Args:
        model: Transformer 모델
        iterator: 학습 데이터 로더
        optimizer: 옵티마이저 (Adam)
        criterion: 손실 함수 (CrossEntropyLoss)
        clip: 그래디언트 클리핑 값

    Returns:
        평균 손실
    """
    # 학습 모드로 설정 (드롭아웃 활성화)
    model.train()

    epoch_loss = 0  # 에폭 총 손실

    # 모든 배치에 대해 반복
    for batch in iterator:
        # 1. 데이터를 디바이스로 이동
        src = batch['SRC'].to(device)  # 소스 (질문)
        trg = batch['TRG'].to(device)  # 타겟 (답변)

        # 2. 그래디언트 초기화 (이전 배치의 그래디언트 제거)
        optimizer.zero_grad()

        # 3. Forward pass: 모델 예측
        # output: (batch_size, tgt_len, vocab_size)
        output, _, _ = model(src, trg)

        # 4. 손실 계산
        # output[:, :-1]: 마지막 토큰 제외 (다음 토큰 예측용)
        # trg[:, 1:]: 첫 토큰 제외 (BOS 제거, 실제 정답)
        # Teacher Forcing: 실제 정답을 입력으로 사용

        # Reshape for loss calculation
        # (batch_size, tgt_len-1, vocab_size) → (batch_size * (tgt_len-1), vocab_size)
        output = output[:, :-1, :].reshape(-1, output.shape[-1])

        # (batch_size, tgt_len-1) → (batch_size * (tgt_len-1))
        trg = trg[:, 1:].reshape(-1)

        # Cross Entropy Loss 계산
        loss = criterion(output, trg)

        # 5. Backward pass: 그래디언트 계산
        loss.backward()

        # 6. 그래디언트 클리핑 (그래디언트 폭발 방지)
        # 그래디언트의 노름이 clip을 넘지 않도록 조정
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)

        # 7. 파라미터 업데이트
        optimizer.step()

        # 배치 손실 누적
        epoch_loss += loss.item()

    # 평균 손실 반환
    return epoch_loss / len(iterator)


def evaluate(model, iterator, criterion):
    """
    모델 평가 (검증/테스트)

    학습과 유사하지만:
    - 그래디언트 계산 안 함 (torch.no_grad)
    - 파라미터 업데이트 안 함
    - 드롭아웃 비활성화 (model.eval)

    Args:
        model: 평가할 모델
        iterator: 검증/테스트 데이터 로더
        criterion: 손실 함수

    Returns:
        평균 손실
    """
    # 평가 모드로 설정 (드롭아웃 비활성화)
    model.eval()

    epoch_loss = 0

    # 그래디언트 계산 비활성화 (메모리 절약, 속도 향상)
    with torch.no_grad():
        for batch in iterator:
            # 데이터 로드
            src = batch['SRC'].to(device)
            trg = batch['TRG'].to(device)

            # Forward pass (그래디언트 계산 안 함)
            output, _, _ = model(src, trg)

            # 손실 계산 (학습과 동일)
            output = output[:, :-1, :].reshape(-1, output.shape[-1])
            trg = trg[:, 1:].reshape(-1)
            loss = criterion(output, trg)

            # 손실 누적
            epoch_loss += loss.item()

    # 평균 손실 반환
    return epoch_loss / len(iterator)


In [258]:
# ===== 모델 학습 시작 =====

# 학습 하이퍼파라미터
N_EPOCHS = 100  # 최대 에폭 수
CLIP = 1  # 그래디언트 클리핑
best_valid_loss = float('inf')  # 최고 검증 손실 (초기값: 무한대)

print("=" * 50)
print("Start Model Training")
print("=" * 50)

# Early Stopping 설정
patience = 5  # 성능 개선 없으면 5 에폭 후 종료
patience_counter = 0  # 카운터 초기화

# 학습 루프
for epoch in tqdm(range(N_EPOCHS), desc="Training in Progress"):
    # 1. 학습
    train_loss = train(model, train_iterator, optimizer, criterion, CLIP)

    # 2. 검증
    valid_loss = evaluate(model, valid_iterator, criterion)

    # 3. Early Stopping 체크
    if valid_loss < best_valid_loss:
        # 검증 손실이 개선됨
        best_valid_loss = valid_loss  # 최고 기록 갱신
        patience_counter = 0  # 카운터 리셋

        # 최고 성능 모델 저장
        torch.save(model.state_dict(), 'best_model.pt')
        print(f'\n[Epoch {epoch+1}] Model Saved! Valid Loss: {valid_loss:.3f}')
    else:
        # 검증 손실이 개선되지 않음
        patience_counter += 1  # 카운터 증가
        print(f'\n[Epoch {epoch+1}] No Improvement ({patience_counter}/{patience})')

        # Patience 초과 시 조기 종료
        if patience_counter >= patience:
            print(f"\nEarly Stopping! (최고 Valid Loss: {best_valid_loss:.3f})")
            break

    # 4. 학습 상태 출력
    if (epoch + 1) % 5 == 0:  # 5 에폭마다 출력
        print(f'\nEpoch: {epoch+1:02}')
        print(f'\tTrain Loss: {train_loss:.3f}')
        print(f'\tValidation Loss: {valid_loss:.3f}')

print("\n" + "=" * 50)
print("학습 완료!")
print("=" * 50)


Start Model Training


Training in Progress:   1%|          | 1/100 [01:04<1:45:48, 64.12s/it]


[Epoch 1] Model Saved! Valid Loss: 3.205


Training in Progress:   2%|▏         | 2/100 [01:54<1:31:45, 56.18s/it]


[Epoch 2] Model Saved! Valid Loss: 2.898


...


[Epoch 19] No Improvement (3/5)


Training in Progress:  20%|██        | 20/100 [17:16<1:10:41, 53.02s/it]


[Epoch 20] No Improvement (4/5)

Epoch: 20
	Train Loss: 1.060
	Validation Loss: 2.133


Training in Progress:  20%|██        | 20/100 [18:14<1:12:56, 54.70s/it]


[Epoch 21] No Improvement (5/5)

Early Stopping! (최고 Valid Loss: 2.122)

학습 완료!


In [259]:
# ===== 테스트 세트 최종 평가 =====

# 저장된 최고 모델 로드
model.load_state_dict(torch.load('best_model.pt'))

# 테스트 세트 평가
test_loss = evaluate(model, test_iterator, criterion)

print(f"\n테스트 손실: {test_loss:.3f}")
print(f"테스트 Perplexity: {np.exp(test_loss):.3f}")  # Perplexity: exp(loss)
# Perplexity: 모델의 불확실성 지표 (낮을수록 좋음)



테스트 손실: 2.099
테스트 Perplexity: 8.157


In [277]:
# ===== 답변 생성 함수 (추론) =====

def generate_response(model, question, vocab, max_length=40):
    """
    주어진 질문에 대해 모델이 답변을 생성

    생성 방식: Greedy Decoding
    - 각 스텝에서 가장 확률이 높은 토큰 선택
    - 빠르지만 최적이 아닐 수 있음

    Args:
        model: 학습된 Transformer 모델
        question: 입력 질문 (문자열)
        vocab: 어휘 사전
        max_length: 최대 생성 길이

    Returns:
        생성된 답변 (문자열)
    """
    # 평가 모드
    model.eval()

    # 질문 전처리 및 토큰화
    question = preprocess_sentence(question)  # 텍스트 정제
    src_ids = [vocab.BOS_ID] + vocab.encode(question) + [vocab.EOS_ID]  # [BOS] + tokens + [EOS]
    src = torch.tensor(src_ids, dtype=torch.long).unsqueeze(0).to(device)  # (1, src_len)

    # 그래디언트 계산 비활성화
    with torch.no_grad():
        # 1. 인코더: 질문 인코딩
        enc_mask = (src == vocab.stoi['<pad>'])  # 패딩 마스크
        encoder_output, _ = model.encoder(src, attention_mask=enc_mask)

        # 2. 디코더: 답변 생성 (자동 회귀)
        # 초기 입력: [BOS] 토큰
        trg_ids = [vocab.BOS_ID]

        # 최대 길이까지 또는 [EOS] 생성까지 반복
        for _ in range(max_length):
            # 현재까지 생성된 타겟 시퀀스
            trg = torch.tensor(trg_ids, dtype=torch.long).unsqueeze(0).to(device)  # (1, cur_len)

            # 인과 마스크 생성 (미래 토큰 마스킹)
            tgt_len = trg.size(1)
            dec_mask = torch.triu(
                torch.ones(tgt_len, tgt_len, dtype=torch.bool, device=device),
                diagonal=1
            )

            # 디코더 forward
            decoder_output, _ = model.decoder(
                trg, encoder_output,
                encoder_attention_mask=enc_mask,
                decoder_causal_mask=dec_mask
            )

            # 최종 출력 레이어
            # (1, cur_len, emb_dim) → (1, cur_len, vocab_size)
            output = model.prediction_head(decoder_output)

            # 마지막 토큰의 예측만 사용
            # (1, vocab_size)
            next_token_logits = output[:, -1, :]

            # 가장 높은 확률의 토큰 선택 (Greedy)
            next_token = next_token_logits.argmax(dim=-1).item()

            # 생성된 토큰을 시퀀스에 추가
            trg_ids.append(next_token)

            # [EOS] 토큰이 생성되면 종료
            if next_token == vocab.EOS_ID:
                break

    # 3. 토큰 ID를 문자열로 디코딩
    # BOS, EOS 제거
    response = vocab.decode(trg_ids[1:-1])  # [BOS]와 [EOS] 제외

    return response


# ===== 예제 추론 =====

# 테스트 질문들
test_questions = [
    "짝사랑이랑 연애하고 싶다",
    "오늘 날씨가 안좋아!",
    "뭐 하고 있나요 하고 메시지 보내고 싶어",
    "고기 먹고 싶어",
    "내가 좋아하는 거 알았는데도 나를 대하는게 변함이 없어.",
    "내가 좋아하는 걸 티냈는데 그 사람은 반응이 없어.",
    "요즘 너무 외로워서 힘들어.",
]

print("\n" + "=" * 50)
print("모델 추론 예제")
print("=" * 50)

# 각 질문에 대해 답변 생성
for question in test_questions:
    response = generate_response(model, question, vocab)
    print(f"\n질문: {question}")
    print(f"답변: {response}")



모델 추론 예제

질문: 짝사랑이랑 연애하고 싶다
답변: 짝사랑이 그렇게 만드네요.

질문: 오늘 날씨가 안좋아!
답변: 마스크 쓰고 좋겠네요.

질문: 뭐 하고 있나요 하고 메시지 보내고 싶어
답변: 어떤 결정이었을텐데 맘고생 많았어요.

질문: 고기 먹고 싶어
답변: 저도 밥 먹고 싶어요.

질문: 내가 좋아하는 거 알았는데도 나를 대하는게 변함이 없어.
답변: 관심을 가지세요.

질문: 내가 좋아하는 걸 티냈는데 그 사람은 반응이 없어.
답변: 서로의 의사가 존중된 것이라면 좋겠어요.

질문: 요즘 너무 외로워서 힘들어.
답변: 위험하네요.


In [261]:
#추가 실험

df = pd.read_csv(file_path)
questions_from_file = df['Q'].tolist()
questions_to_test = questions_from_file[50:80]
for question in questions_to_test:
    response = generate_response(model, question, vocab)
    print(f"\nQ (from file): {question}")
    print(f"A (model reply): {response}")

print("\n" + "=" * 50)
print("CSV 파일 테스트 완료.")
print("=" * 50)


Q (from file): 감 말랭이 먹어야지
A (model reply): 맛있게 드세요.

Q (from file): 감기 같애
A (model reply): 병원가세요.

Q (from file): 감기 걸린 것 같아
A (model reply): 이젠 그럴 때가 있죠.

Q (from file): 감기 기운이 있어
A (model reply): 운동을 해보는 건 어떨까요.

Q (from file): 감기 들 거 같애
A (model reply): 병원가세요.

Q (from file): 감기가 오려나
A (model reply): 병원가세요.

Q (from file): 감기약이 없어
A (model reply): 병원가세요.

Q (from file): 감기인거 같애
A (model reply): 병원가세요.

Q (from file): 감미로운 목소리 좋아
A (model reply): 스트레스가 심하겠네요.

Q (from file): 감정이 쓰레기통처럼 엉망진창이야
A (model reply): 상처받았으면 인연이라면 상처가 필요하겠네요.

Q (from file): 감정컨트롤을 못하겠어
A (model reply): 스스로 존중하고 중하고 중하세요.

Q (from file): 감정컨트롤이 안돼
A (model reply): 스스로 존중하고 중하고 중하세요.

Q (from file): 감히 나를 무시하는 애가 있어
A (model reply): 상처가 될 거예요.

Q (from file): 갑자기 나쁜 생각이 막 들더라
A (model reply): 상처받지 않았으면 좋겠어요.

Q (from file): 갑자기 눈물 나
A (model reply): 고마워질 거예요.

Q (from file): 갑자기 물어봐서 당황했어
A (model reply): 갑작스럽겠네요.

Q (from file): 갑자기 불편한 사이가 된 거 같아
A (model reply): 제가 있잖아요.

Q (from file): 강렬한 첫인상 남겨야 하는데
A (